# Verify Credential

This notebook illustrates how a VC signed using a BIP 322 signature could be parsed and verified by a verifier.

## Notebook Author

* Will Abramson
* [Legendary Requirements](http://legreq.com/)
* Contact: will@legreq.com

## Acknowledgements

This work was funded by Ryan Grant and [Digital Contract Design](https://contract.design/). Thanks also go to Joe Andrieu, Kalle Alm, Pieter Wuille and Jimmy Song for engaging with and supporting various aspects of this work.

## Steps

1. Parse signature from signed VC
2. Parse/resolve bitcoin address used the sign vc
3. Remove proof from signed VC 
4. Canonicalize VC
5. Verify signatue on VC

In [1]:
cd ..

/home/will/work/LegendaryRequirements/clients/dcd/bip0322-sigs


## 1. Parse signature from signed VC

**YOU SHOULD COPY THE SIGNED VC PRODUCED IN the [issue_credential](http://localhost:8888/notebooks/vc/issue_credential.ipynb) NOTEBOOK**

In [2]:
signed_vc ={"@context": ["https://www.w3.org/2018/credentials/v1", "https://www.w3.org/2018/credentials/examples/v1"], "id": "http://example.edu/credentials/1872", "type": ["VerifiableCredential", "AlumniCredential"], "issuer": "https://example.edu/issuers/565049", "issuanceDate": "2010-01-01T19:23:24Z", "credentialSubject": {"id": "did:example:ebfeb1f712ebc6f1c276e12ec21", "alumniOf": {"id": "did:example:c276e12ec21ebfeb1f712ebc6f1", "name": [{"value": "Example University", "lang": "en"}, {"value": "Exemple d'Universit\u00e9", "lang": "fr"}]}}, "proof": {"type": "BIP322SimpleSignature2022", "created": "2022-05-23T21:19:10Z", "proofPurpose": "assertionMethod", "verificationMethod": "https://example.edu/issuers/565049#key-1", "bip322_sig": "AkgwRQIhAKgZ4oVu51lwBxQ+u0yz1xHBg60ovueTTvU8Wl5RkrhoAiAfY5WS+ghkFmQZzhLcGN6wSAjlQeLbBYgPi1Ve3hvnswEhA8iCJXTXxwVa3nx91ZjdWvR+cjVLVuP6HiwsPPAl9s8x"}}


In [3]:
proof = signed_vc["proof"]
bip322_sig = proof["bip322_sig"]

In [4]:
print("bip322 signature in VC : \n", bip322_sig)

bip322 signature in VC : 
 AkgwRQIhAKgZ4oVu51lwBxQ+u0yz1xHBg60ovueTTvU8Wl5RkrhoAiAfY5WS+ghkFmQZzhLcGN6wSAjlQeLbBYgPi1Ve3hvnswEhA8iCJXTXxwVa3nx91ZjdWvR+cjVLVuP6HiwsPPAl9s8x


## 2. Resove Bitcoin address used to sign VC

This could either be from:

- TxRef
- Secp public key and script type
- Script and public key
- UTXO
- A did:btcr(2)?

In [5]:
print("This needs to be something that can be resolved to a BTC address :", signed_vc["issuer"])

This needs to be something that can be resolved to a BTC address : https://example.edu/issuers/565049


In [6]:
## TODO: this should be contained within the VC or retrievable through did resolution.
## Currently just copied from issue_credential
address = 'bc1qz52z3pe9fg3qxv9n6yhxgj7rcn8wsvpq56v9ck'

## 3. Remove proof from signed_vc payload

In [7]:
vc = signed_vc
del vc["proof"]

vc

{'@context': ['https://www.w3.org/2018/credentials/v1',
  'https://www.w3.org/2018/credentials/examples/v1'],
 'id': 'http://example.edu/credentials/1872',
 'type': ['VerifiableCredential', 'AlumniCredential'],
 'issuer': 'https://example.edu/issuers/565049',
 'issuanceDate': '2010-01-01T19:23:24Z',
 'credentialSubject': {'id': 'did:example:ebfeb1f712ebc6f1c276e12ec21',
  'alumniOf': {'id': 'did:example:c276e12ec21ebfeb1f712ebc6f1',
   'name': [{'value': 'Example University', 'lang': 'en'},
    {'value': "Exemple d'Université", 'lang': 'fr'}]}}}

## 4. Canonicalize VC payload

In [8]:
import json
from buidl.helper import str_to_bytes
from buidl.script import address_to_script_pubkey

In [9]:
address_to_script_pubkey(address)

OP_0 15142887254a220330b3d12e644bc3c4cee83020 

In [10]:
vc_payload = json.dumps(vc)

In [11]:
str_to_bytes(vc_payload)

b'{"@context": ["https://www.w3.org/2018/credentials/v1", "https://www.w3.org/2018/credentials/examples/v1"], "id": "http://example.edu/credentials/1872", "type": ["VerifiableCredential", "AlumniCredential"], "issuer": "https://example.edu/issuers/565049", "issuanceDate": "2010-01-01T19:23:24Z", "credentialSubject": {"id": "did:example:ebfeb1f712ebc6f1c276e12ec21", "alumniOf": {"id": "did:example:c276e12ec21ebfeb1f712ebc6f1", "name": [{"value": "Example University", "lang": "en"}, {"value": "Exemple d\'Universit\\u00e9", "lang": "fr"}]}}}'

## 5. Verify bip322 signature on VC

In [13]:
from src.message import verify_message

### NOTE: the output `"mismatch between length and consumed bytes 88 vs 31"` is expected" and occurs when attempting to parse the signature as a full Bitcoin transaction. See is_full_signature in [src/message.py](../../edit/src/message.py) and the [script.py](https://github.com/buidl-bitcoin/buidl-python/blob/e62e0b9fd4271473cf641cde0fe366e65c787403/buidl/script.py#L109) file from Buidl Python

In [15]:

verification_result = verify_message(address, bip322_sig, vc_payload)

print("Signature is verified : ", verification_result)

mismatch between length and consumed bytes 88 vs 31
Signature is not full
Signature is verified :  True
